# Multi-tube design example for orthogonal systems (simple)

Design N orthogonal strand displacement gates. 

See the accompanying design specification (PDF file). See also the LaTeX spec file that you can edit to make your own design specs in a standardized format.

This is a 1-step reaction. To design N orthogonal gates, there are 2 elementary step tubes per gate plus 1 global crosstalk tube. 

Target test tubes: 
- (Step 0: Reactants) x N orthogonal systems
- (Step 1: Products) x N orthogonal systems
- Global crosstalk

Material: RNA    
Temperature: 23 C

In [1]:
# Import Python NUPACK module
from nupack import *

In [2]:
# Define physical model
my_model = Model(material='rna', celsius=23)

# Set number of orthogonal systems
systems = 4  

# Initialize working variables
crosstalk_targets = {}
crosstalk_excludes = []
step_tubes = []

# Loop over orthogonal systems
for i in range(systems):
    
    # Define sequence domains
    da = Domain('N10', name=['da', i])
    db = Domain('N8', name=['db', i])

    # Define strands containing these domains
    sX = TargetStrand([da, db], name=['sX', i])
    sA = TargetStrand([~db, ~da], name=['sA', i])
    sB = TargetStrand([da], name=['sB', i])  # ~dgate is the reverse complement of dgate
    sA_toe = TargetStrand([~db], name=['sA_toe', i])
    
    # Define target complexes
    cX = TargetComplex([sX], 'U18', name=['cX', i])
    cAB = TargetComplex([sA, sB], 'U8D10+', name=['cAB', i])
    cXA = TargetComplex([sX, sA], 'D18+', name=['cXA', i])
    cB = TargetComplex([sB], 'U10', name=['cB', i])
    cA_toe = TargetComplex([sA_toe], 'U8', name=['cA_toe', i])
    
    # Define 2 elementary step tubes for each orthogonal system
    step0_tube = TargetTube(on_targets={cX: 1e-08, cAB: 1e-08}, 
                           off_targets=SetSpec(max_size=2, exclude=[cXA]), name=['reactants', i])
    step1_tube = TargetTube(on_targets={cXA: 1e-08, cB: 1e-08}, 
                          off_targets=SetSpec(max_size=2), name=['products', i])
    step_tubes += [step0_tube, step1_tube]
    
    # Store contributions to global crosstalk tube
    crosstalk_targets.update({cX: 1e-08, cAB: 1e-08, cA_toe: 1e-08, cB:1e-8})
    crosstalk_excludes += [cXA, [sX, sA_toe]]
    
# Define 1 global crosstalk tube 
crosstalk_tube = TargetTube(on_targets=crosstalk_targets, 
                        off_targets=SetSpec(max_size=2, exclude=crosstalk_excludes), name='crosstalk')

# Define tube set    
my_tubes = step_tubes + [crosstalk_tube]    
    
# Define defect weight of N for global crosstalk tube 
# (maintain balance between elementary step tubes and global crosstalk tube as number of 
# orthogonal systems increases) 
my_weights = Weights(my_tubes)
my_weights[:,:,:,crosstalk_tube] *= systems

# Set a stop condition of 2% 
# Set seed for random number generation to get a reproducible result for this demo
my_options = DesignOptions(f_stop=0.02, seed=93)

# Define and run the test tube design job
my_design = tube_design(tubes=my_tubes, defect_weights=my_weights, model=my_model, options=my_options)
my_results = my_design.run(trials=1)[0]

# Display the design results
my_results

Domain,Sequence
da[0],CAUACUCCGA
da[0]*,UCGGAGUAUG
da[1],CACAUGAUUC
da[1]*,GAAUCAUGUG
da[2],GAACAAGCUA
da[2]*,UAGCUUGUUC
da[3],CCUUUCUCUC
da[3]*,GAGAGAAAGG
db[0],AACCCAAC
db[0]*,GUUGGGUU
